In [30]:
import dash
from dash.dependencies import Input, Output
from dash import dcc, html
import plotly.express as px
import pandas as pd

import webbrowser
from threading import Timer

In [31]:
df = pd.read_csv('Data/clean_fall_data.csv')
df.head()

,Unnamed: 0,distance (cm),pressure,hrv (bpm),blood sugar level (mg/dL),spo2,accelerometer,decision
0,0,25.540,1,101.396,61.080,87.770,1,1
1,1,2.595,2,110.190,20.207,65.190,1,2
2,2,68.067,0,87.412,79.345,99.345,0,0
3,3,13.090,1,92.266,36.180,81.545,1,1
4,4,69.430,0,89.480,80.000,99.990,0,0


In [32]:
df = df.drop(columns='Unnamed: 0')
df.head()

,distance (cm),pressure,hrv (bpm),blood sugar level (mg/dL),spo2,accelerometer,decision
0,25.540,1,101.396,61.080,87.770,1,1
1,2.595,2,110.190,20.207,65.190,1,2
2,68.067,0,87.412,79.345,99.345,0,0
3,13.090,1,92.266,36.180,81.545,1,1
4,69.430,0,89.480,80.000,99.990,0,0


In [19]:
# Initialize the app
app = dash.Dash(__name__)

In [33]:
# Initialize the app
app = dash.Dash(__name__)

# Define the app layout with a 4x4 grid layout
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(
                    children="Fall Prediction Analytics", className="header-title"
                ),
                html.P(
                    children=(
                       'Analyzing the trends in variables in relation to elderly'
                        'people falling, tripping, or not falling'
                    ),
                    className="header-description",
                ),
            ],
            className="header"),
    html.Div([
        html.Label("Select Feature:"),
        dcc.Dropdown(
            id='feature-selector',
            options=[{'label': i, 'value': i} for i in df.columns if i not in ['Unnamed: 0', 'decision']],
            value='distance (cm)'
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),
    
    # Define the grid layout for the visuals
    html.Div([
        dcc.Graph(id='scatter-plot', style={'display': 'inline-block', 'width': '48%'}),
        dcc.Graph(id='histogram', style={'display': 'inline-block', 'width': '48%'}),
        dcc.Graph(id='boxplot', style={'display': 'inline-block', 'width': '48%'}),
        dcc.Graph(id='heatmap', style={'display': 'inline-block', 'width': '48%'}),
    ]),
])

# Define the callback for updating plots
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('histogram', 'figure'),
     Output('boxplot', 'figure'),
     Output('heatmap', 'figure')],
    [Input('feature-selector', 'value')]
)
def update_plots(selected_feature):
    scatter_fig = px.scatter(
        df, x=selected_feature, y='decision',
        title=f"Scatter Plot: 'decision' vs {selected_feature}")

    hist_fig = px.histogram(
        df, x=selected_feature, color='decision',
        title=f"Histogram of {selected_feature} colored by 'decision'")

    box_fig = px.box(
        df, y=selected_feature, x='decision',
        title=f"Box Plot of {selected_feature} grouped by 'decision'")

    # Compute the correlation matrix and use it for the heatmap
    heatmap_fig = px.imshow(df.corr(), text_auto=True, aspect="auto",
                            title="Heatmap of Feature Correlations")

    return scatter_fig, hist_fig, box_fig, heatmap_fig


# Run the app
port = 5000 # or simply open on the default `8050` port

def open_browser():
	webbrowser.open_new("http://localhost:{}".format(port))

if __name__ == '__main__':
    Timer(1, open_browser).start();
    app.run_server(debug=True, port=port)